### [MAPLEAF.ENV](https://henrystoldt.github.io/MAPLEAF/ENV/index.html)
For the rocket to compute the forces it experiences it needs to know about its environment.

Environmental modelling in MAPLEAF is divided into modelling of:
- Mean winds -> MAPLEAF.ENV.MeanWindModelling
- Air turbulence -> MAPLEAF.ENV.TurbulenceModelling
- Air properties (temp, pressure, density, viscosity) -> MAPLEAF.ENV.AtmosphereModelling
- Gravity -> MAPLEAF.ENV.EarthModelling
- Earth's surface and rotation -> MAPLEAF.ENV.EarthModelling

Each of these modules defines a 'factory' function, which reads info from a SimDefinition, and then returns an appropriate model object.
            
All the models end up tied together by the Environment class, which is what the rocket communicates with to figure out what its flying through.

**Mean Wind Modelling** can take several forms:
- Constant wind (Deterministic)
- Hellman (Deterministic)
- Custom Profile (Deterministic)
- SampledGroundWind (Probabilistic)
- SampledRadioSondeData (Probabilistic)

In [1]:
from MAPLEAF.ENV import meanWindModelFactory
from MAPLEAF.IO import SimDefinition
import matplotlib.pyplot as plt

simDef = SimDefinition("MAPLEAF/Examples/Simulations/Wind.mapleaf")

### Remove keys that make probabilistic models deterministic
randomSeedKeys = simDef.findKeysContaining("randomSeed")
print(randomSeedKeys)
for key in randomSeedKeys:
    simDef.removeKey(key)
    
# Try out constant model (default)
simDef.setValue("Environment.MeanWindModel", "Constant")

windModel = meanWindModelFactory(simDef, silent=True)

def plotWinds(model):
    altitudes = list(range(0, 50000, 1000))
    winds = [ model.getMeanWind(a).length() for a in altitudes]
    plt.plot(altitudes, winds)
    plt.xlabel("Altitude(m)")
    plt.ylabel("Wind Speed (m/s)")
    
plotWinds(windModel)

UnboundLocalError: local variable 'str' referenced before assignment

In [ ]:
# Hellman is a simple model for very low altitudes
simDef.setValue("Environment.MeanWindModel", "Hellman")
windModel = meanWindModelFactory(simDef, silent=True)
plotWinds(windModel)

In [ ]:
# Sampling wind probability ditributions from wind rose datas
simDef.setValue("Environment.MeanWindModel", "SampledGroundWindData")
windModel = meanWindModelFactory(simDef, silent=True)
plotWinds(windModel)
print(windModel.getMeanWind(100))

In [ ]:
# Sampling entire radio sonde wind profiles
simDef.setValue("Environment.MeanWindModel", "SampledRadioSondeData")
windModel = meanWindModelFactory(simDef, silent=True)
plotWinds(windModel)

**Turbulence Modelling** has three general possibilities:
- Pink Noise Models (probabilistic, depends on mean wind)
- Custom gust (deterministic, doesn't depend on mean wind)
- None (default)

In [ ]:
from MAPLEAF.ENV import turbulenceModelFactory
simDef.setValue("Environment.MeanWindModel", "Constant")
meanWindModel = meanWindModelFactory(simDef, silent=True)

simDef.setValue("Environment.TurbulenceModel", "PinkNoise3D")
turbModel = turbulenceModelFactory(simDef, silent=True)

def plotTurb(turbModel, meanWindModel):
    altitudes = list(range(0, 50000, 1000))
    times = [ x/50 for x in range(100) ]
    meanWinds = [ meanWindModel.getMeanWind(a) for a in altitudes ]
    turbComponent = [ turbModel.getTurbVelocity(altitude=a, meanWindVelocity=m, time=t).length() for a,m,t in zip(altitudes, meanWinds, times)]
    
    plt.plot(altitudes, turbComponent)
    plt.xlabel("Altitude(m)")
    plt.ylabel("Turbulent Wind Speed (m/s)")
    
plotTurb(turbModel, meanWindModel)

In [ ]:
simDef.setValue("Environment.TurbulenceModel", "customSineGust")
turbModel = turbulenceModelFactory(simDef, silent=True)
plotTurb(turbModel, meanWindModel)

In [ ]:
print(simDef)